In [1]:
using Knet   
using Compat,GZip # Helpers to read the MNIST (Like lab-2)
Pkg.add("Plots")
using Plots
Pkg.add("JLD")
using JLD

INFO: Package Plots is already installed
INFO: METADATA is out-of-date — you may not have the latest version of Plots
INFO: Use `Pkg.update()` to get the latest versions of your packages
INFO: Package JLD is already installed
INFO: METADATA is out-of-date — you may not have the latest version of JLD
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [2]:
include(Pkg.dir("Knet","data","mnist.jl"))#include mnist related functions
xtrn,ytrn,xtst,ytst =mnist()#data ready to use in convolutional neural networks
dtrn=minibatch(xtrn, ytrn, 100; xtype=KnetArray{Float32})#returns list of tuples of (x,y)
dtst=minibatch(xtst,ytst,100;xtype=KnetArray{Float32})

INFO: Loading MNIST...


Knet.MB(Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], UInt8[0x07 0x02 … 0x05 0x06], 100, 10000, false, 1:10000, [28, 28, 1, 10000], [10000], Knet.KnetArray{Float32,N} where N, Array{UInt8,1})

In [3]:
function weights(A;atype=KnetArray{Float32})
    w = Array{Any}(length(A)-1)
    input=A[1]
    for i=2:1:length(A)
        x1,x2,cx=input
        a=A[i]
        if typeof(a)==Int64          
            w[i-1]=[xavier(a,x1*x2*cx), zeros(a,1)]
            input=[a,1,1]
        else
            d1,d2,cy=a
            w[i-1]=[xavier(d1,d2,cx,cy), zeros(1,1,cy,1)]
            input=[div((x1-d1+1),2),div((x1-d1+1),2),cy]
        end
    end
    return map(b->map(a->convert(atype,a),b),w)
end

weights (generic function with 1 method)

In [26]:
#https://github.com/denizyuret/Knet.jl/blob/master/examples/dl-tutorial/mnist.ipynb
softmax(w,data,predfun;actfun=sigm) = mean(softmax(w,x,y,predfun,actfun) for (x,y) in data)
softmax(w,x,y,predfun; actfun=sigm, o...) = nll(predfun(w,x,actfun;o...),y)
softgrad = grad(softmax)
zeroone(w,data,model) = 1 - accuracy(w,data,model)

zeroone (generic function with 1 method)

In [12]:
function predict(weight,x,actfun=sigm)
    for i in 1:1:size(weight,1)-1
        w=weight[i]
        if ndims(w[1])==4
            x = conv4(w[1],x) .+ w[2]
            x = pool(actfun.(x))
        elseif ndims(w[1])==2
            x=mat(x)
            x =actfun.( w[1] * x .+ w[2] )
        end
    end
    x=weight[end][1]*mat(x).+weight[end][2]
       return x
end

predict (generic function with 2 methods)

In [14]:
function train(w,dtrn, actfun=sigm, optim=nothing, epochs=100, lr=0.15)
 Weights = Any[deepcopy(w)]
 for epoch in 1:epochs
    for (x,y) in dtrn  # dtrn is a list of minibatches
        g = softgrad(w, x, y,predict,actfun=actfun)
            if(optim==nothing)
                update!(w,g,lr=lr)
            else
                update!(w,g,optim)
            end
    end
        push!(Weights,deepcopy(w))

    end
    return Weights
end

train (generic function with 5 methods)

In [27]:
##Question 5
setseed(13)
myWeights=weights([[28,28,1],[5,5,3],10],atype=KnetArray{Float32})
@time Weights=train(myWeights,dtrn, optim=nothing) # 65.1 s
@time trnloss1 = [ softmax(w,dtrn,predict) for w in Weights ]   # 32.4 s
@time tstloss1 = [ softmax(w,dtst,predict) for w in Weights ]   # 5.4 s
@time trnerr1 = [ zeroone(w,dtrn,predict) for w in Weights ]    # 35.9 s
@time tsterr1 = [ zeroone(w,dtst,predict) for w in Weights ]    # 6.1 s
save("data.jld","trnloss",trnloss1, "tstloss", tstloss1,"trnerr", trnerr1,"tsterrr",tsterr1)

 65.149150 seconds (51.36 M allocations: 19.847 GiB, 4.14% gc time)
 32.478828 seconds (14.02 M allocations: 18.307 GiB, 5.76% gc time)
  5.396918 seconds (2.31 M allocations: 3.050 GiB, 5.86% gc time)
 34.641949 seconds (13.95 M allocations: 18.835 GiB, 5.60% gc time)
  5.773033 seconds (2.28 M allocations: 3.137 GiB, 5.69% gc time)


LoadError: [91mUndefVarError: trnloss not defined[39m

In [29]:
Pkg.add("Plots")
using Plots
plotly() # Choose the Plotly.jl backend for web interactivity
plot([trnloss1 tstloss1],ylim=(0,0.15),
    labels=[:trn :tst],xlabel="Epochs",ylabel="Loss")


INFO: Package Plots is already installed
INFO: METADATA is out-of-date — you may not have the latest version of Plots
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [30]:
plot([trnerr1 tstloss1],ylim=(0,0.15),
    labels=[:trn :tst],xlabel="Epochs",ylabel="Error")


In [31]:
##Question 6
setseed(13)
myWeights=weights([[28,28,1],[5,5,3],10],atype=KnetArray{Float32})
myOptim=map(a->map(x->Adam(),a),myWeights);
@time Weights=train(myWeights,dtrn, optim=myOptim) # 82.1 s
@time trnloss2 = [ softmax(w,dtrn,predict) for w in Weights ]   # 32.3 s
@time tstloss2 = [ softmax(w,dtst,predict) for w in Weights ]   # 5.4 s
@time trnerr2 = [ zeroone(w,dtrn,predict) for w in Weights ]    # 35.0 s
@time tsterr2 = [ zeroone(w,dtst,predict) for w in Weights ]    # 5.9 s
save("data.jld","trnloss",trnloss2, "tstloss", tstloss2,"trnerr", trnerr2,"tsterrr",tsterr2)

 83.627556 seconds (63.98 M allocations: 20.156 GiB, 5.86% gc time)
 34.247107 seconds (13.83 M allocations: 18.297 GiB, 8.66% gc time)
  5.782654 seconds (2.31 M allocations: 3.050 GiB, 8.75% gc time)
 36.685688 seconds (13.64 M allocations: 18.819 GiB, 8.22% gc time)
  6.212180 seconds (2.28 M allocations: 3.137 GiB, 8.18% gc time)


In [32]:
Pkg.add("Plots")
using Plots
plotly() # Choose the Plotly.jl backend for web interactivity
plot([trnloss2 tstloss2],ylim=(0,0.15),
    labels=[:trn :tst],xlabel="Epochs",ylabel="Loss")


INFO: Package Plots is already installed
INFO: METADATA is out-of-date — you may not have the latest version of Plots
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [35]:
plot([trnerr2 tstloss2],ylim=(0,0.15),
    labels=[:trn :tst],xlabel="Epochs",ylabel="Error")


In [34]:
myWeights=weights([[28,28,1],[5,5,20],[5,5,30],30,10],atype=KnetArray{Float32})
@time Weights=train(myWeights,dtrn, optim=nothing) # 170.05 s
@time trnloss3 = [ softmax(w,dtrn,predict) for w in Weights ]   # 67.2 s
@time tstloss3 = [ softmax(w,dtst,predict) for w in Weights ]   # 11.2 s
@time trnerr3 = [ zeroone(w,dtrn,predict) for w in Weights ]    # 67.8 s
@time tsterr3 = [ zeroone(w,dtst,predict) for w in Weights ]    # 11.4 s
save("data.jld","trnloss",trnloss3, "tstloss", tstloss3,"trnerr", trnerr3,"tsterrr",tsterr3)

169.395471 seconds (91.07 M allocations: 21.607 GiB, 3.94% gc time)
 67.206451 seconds (23.53 M allocations: 18.678 GiB, 4.08% gc time)
 11.235890 seconds (3.93 M allocations: 3.114 GiB, 4.15% gc time)
 67.564164 seconds (23.34 M allocations: 19.200 GiB, 4.16% gc time)
 11.316496 seconds (3.89 M allocations: 3.201 GiB, 4.19% gc time)


In [36]:
plot([trnloss3 tstloss3],ylim=(0,0.15),
    labels=[:trn :tst],xlabel="Epochs",ylabel="Loss")

In [37]:
plot([trnerr3 tstloss3],ylim=(0,0.15),
    labels=[:trn :tst],xlabel="Epochs",ylabel="Error")

In [38]:
myWeights=weights([[28,28,1],[5,5,20],[5,5,30],30,10],atype=KnetArray{Float32})
@time Weights=train(myWeights,dtrn, actfun=relu, optim=nothing) # 166.05 s
@time trnloss4 = [ softmax(w,dtrn,predict,actfun=relu) for w in Weights ]   # 67.2 s
@time tstloss4 = [ softmax(w,dtst,predict,actfun=relu) for w in Weights ]   # 11.2 s

save("data.jld","trnloss",trnloss4, "tstloss", tstloss4)

166.427726 seconds (91.00 M allocations: 21.604 GiB, 2.68% gc time)
 67.015225 seconds (23.61 M allocations: 18.682 GiB, 4.25% gc time)
 11.167448 seconds (3.93 M allocations: 3.113 GiB, 4.29% gc time)


In [39]:
plot([trnloss4 tstloss4],ylim=(0,0.15),
    labels=[:trn :tst],xlabel="Epochs",ylabel="Loss")